### model training

In [10]:
import pickle
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import time

In [2]:
## load the data
embedding_matrix = pickle.load(open("/Users/brye/_/_code/_projects/sentiment-analysis-in-disaster-related-tweets/src/embedding_matrix.p", "rb"))
data = pickle.load(open("/Users/brye/_/_code/_projects/sentiment-analysis-in-disaster-related-tweets/src/data.p", "rb"))
training_data = pickle.load(open("/Users/brye/_/_code/_projects/nlp-in-disaster-prediction/src/train.p", "rb"))

MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 200 
num_of_words = 16343

In [3]:
train = data[:len(training_data)]
test = data[len(training_data):]
X_train,X_test,y_train,y_test=train_test_split(train,training_data['target'].values,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (6471, 50)
Shape of Validation  (1142, 50)


In [13]:
name = "1-layer-lstm-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(name))

model1=Sequential()
model1.name = name

embedding=Embedding(num_of_words,EMBEDDING_DIM,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_SEQUENCE_LENGTH,trainable=False)

model1.add(embedding)
model1.add(SpatialDropout1D(0.2))
model1.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model1.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=1e-5)

model1.compile(loss='binary_crossentropy', optimizer=optimzer, metrics=['accuracy'])

In [14]:
model1.summary()

Model: "1-layer-lstm-1609333935"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 200)           3268600   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 50, 200)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                67840     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 3,336,505
Trainable params: 67,905
Non-trainable params: 3,268,600
_________________________________________________________________


In [16]:
history1 = model1.fit(X_train, y_train, batch_size=4,epochs=15, validation_data=(X_test,y_test), callbacks=[tensorboard])

Train on 6471 samples, validate on 1142 samples
Epoch 1/15
6471/6471 [==============================] - 149s 23ms/step - loss: 0.6912 - accuracy: 0.5733 - val_loss: 0.6905 - val_accuracy: 0.5473
Epoch 2/15
6471/6471 [==============================] - 146s 23ms/step - loss: 0.6800 - accuracy: 0.5772 - val_loss: 0.6397 - val_accuracy: 0.5806
Epoch 3/15
6471/6471 [==============================] - 147s 23ms/step - loss: 0.5993 - accuracy: 0.7008 - val_loss: 0.5327 - val_accuracy: 0.7618
Epoch 4/15
6471/6471 [==============================] - 151s 23ms/step - loss: 0.5754 - accuracy: 0.7320 - val_loss: 0.5170 - val_accuracy: 0.7627
Epoch 5/15
6471/6471 [==============================] - 146s 23ms/step - loss: 0.5580 - accuracy: 0.7416 - val_loss: 0.5065 - val_accuracy: 0.7697
Epoch 6/15
6471/6471 [==============================] - 146s 23ms/step - loss: 0.5547 - accuracy: 0.7407 - val_loss: 0.5001 - val_accuracy: 0.7767
Epoch 7/15
6471/6471 [==============================] - 146s 23ms/step

In [17]:
model1.save("1-layer-lstm-1609333935")